# Part #1: Train an image classification model with Azure Machine Learning

Based on the Train an Image Classification model tutorials provided [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-train-models-with-aml) and [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-keras) by Microsoft.

## Set up your development environment

All the setup for your development work can be accomplished in a Python notebook.  Setup includes:

* Importing Python packages
* Connecting to a workspace to enable communication between your local computer and remote resources
* Creating an experiment to track all your runs
* Creating a remote compute target to use for training

### Import packages

Import Python packages you need in this session. Also display the Azure Machine Learning SDK version.

In [27]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.53


### Connect to workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `ws`.

In [28]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

azuremeetupml	northeurope	azuremeetuprg	northeurope


### Create experiment

Create an experiment to track the runs in your workspace. A workspace can have muliple experiments. 

In [3]:
experiment_name = 'azuremeetup-ml-exp'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the am-ml-compute with that name is already in your workspace the code will skip the creation process.

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "am-ml-compute")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 1)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses a GPU-backed VM to improve the training performance of the Deep Neural Net
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. am-ml-compute


### Extract and upload our dataset

Shared on Google Drive at: https://drive.google.com/open?id=10euer4DeAdHIO5s1Ryn4AtiBnh6rUQiK. 

First, upload the dataset into the Workspace.

Now make the data accessible remotely by uploading that data from the Workspace into Azure so it can be accessed for remote training. The datastore is a convenient construct associated with your workspace for you to upload/download data, and interact with it from your remote compute targets. It is backed by Azure blob storage account.

The dataset is uploaded into a directory named `dataset` at the root of the datastore.

In [5]:
from zipfile import ZipFile


data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

derivedFilename = os.path.join(data_folder, 'dataset.zip')
zipObj = ZipFile(os.path.join(data_folder, 'dataset.zip'), 'r')

zipObj.extractall('data')

KeyboardInterrupt: 

In [29]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob azuremeetupml5342001805 azureml-blobstore-00cedc23-0c82-4de0-992d-25bf8046c0e3


In [6]:
ds.upload(src_dir=data_folder, target_path='dataset', overwrite=True, show_progress=True)

AzureBlob azuremeetupml5342001805 azureml-blobstore-00cedc23-0c82-4de0-992d-25bf8046c0e3
Uploading an estimated of 4501 files
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/dataset.zip
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1001.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1002.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1003.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1004.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1005.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1006.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1007.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1008.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew

Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1057.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1058.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1059.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1060.jpg
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1061.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1001.jpg, 31 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1062.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1024.jpg, 32 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1063.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/ca

Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1067.jpg, 74 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1105.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1061.jpg, 75 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1106.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1057.jpg, 76 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1107.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1070.jpg, 77 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1108.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1

Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1111.jpg, 111 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1142.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1113.jpg, 112 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1143.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1121.jpg, 113 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1144.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1127.jpg, 114 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1145.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/c

Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1159.jpg, 154 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1185.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1147.jpg, 155 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1186.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1149.jpg, 156 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1187.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1164.jpg, 157 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/cat.1188.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/test/cat/c

...

Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/514150441_92bbfd7095.jpg, 4379 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/528000398_a05e327988.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/520977373_16597b0b6f.jpg, 4380 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/528008203_cc35822235.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/525526005_4628fb1bc4.jpg, 4381 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/528253562_29bb567cac.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/522215903_3c5ae7d806.jpg, 4382 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fl

Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/531474745_51db091872.jpg, 4412 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/65781336_09372a353c.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/6010232_a80e494813.jpg, 4413 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/65961348_91fc4d3350.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/59458898_4da0582ade.jpg, 4414 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/66858119_05302b5db2.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/54546-Joey-Meerkat.jpg, 4415 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16

Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/7023440_d52a4f4bf5.jpg, 4444 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/81208380_3e52e872d3.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/79797306_7fbe3cb8ff.jpg, 4445 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/81213589_26653cde6c.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/6959220_5d097a4a31.jpg, 4446 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/8234763_8b14e64e9c.jpg
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/80287903_78042b4fa7.jpg, 4447 files out of an estimated total of 4501
Uploading /mnt/azmnt/code/Users/live.com#andrew.fletcher16/m

Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/9692063_618ee7abc3.jpg, 4480 files out of an estimated total of 4501
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/81213589_26653cde6c.jpg, 4481 files out of an estimated total of 4501
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/81208373_233b5e5480.jpg, 4482 files out of an estimated total of 4501
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/83581375_eb6ffc63c8.jpg, 4483 files out of an estimated total of 4501
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/82454667_59519d5181.jpg, 4484 files out of an estimated total of 4501
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/meerkat/93190608_91ca66f9b0.jpg, 4485 files out of an estimated total of 4501
Uploaded /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/data/train/m

$AZUREML_DATAREFERENCE_3e9dd524035a4d679399d9ab524d4411

## Train on a remote cluster

For this task, submit the job to the remote training cluster you set up earlier.  To submit a job you:
* Create a directory
* Create a training script
* Create an estimator object
* Submit the job 

### Create a directory

Create a directory to deliver the necessary code from your computer to the remote resource.

In [6]:
import os
script_folder = os.path.join(os.getcwd(), "meetup-training")
os.makedirs(script_folder, exist_ok=True)

### Create a training script

To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train.py` in the directory you just created. 

In [82]:
%%writefile $script_folder/train.py

# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import numpy as np
import argparse
import os

import matplotlib.pyplot as plt

import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Activation
from keras.optimizers import RMSprop
from keras.callbacks import Callback
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

from azureml.core import Run

print("Keras version:", keras.__version__)
print("Tensorflow version:", tf.__version__)

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')

args = parser.parse_args()

data_folder = args.data_folder
train_folder = os.path.join(data_folder, 'train')
test_folder = os.path.join(data_folder, 'test')

n_outputs = 3
n_epochs = 20
batch_size = 32
learning_rate = 0.001
img_size = 150

print('dataset is stored here:', data_folder)
print('training dataset is stored here:', train_folder)
print('test dataset is stored here:', test_folder)

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_folder, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical', subset='training')
validation_generator = train_datagen.flow_from_directory(train_folder, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical', subset='validation')
test_generator = test_datagen.flow_from_directory(test_folder, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')

label_map = (train_generator.class_indices)

print(label_map)

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# start an Azure ML run
run = Run.get_context()

class LogRunMetrics(Callback):
    # callback at the end of every epoch
    def on_epoch_end(self, epoch, log):
        # log a value repeated which creates a list
        run.log('Loss', log['loss'])
        run.log('Accuracy', log['acc'])
        
history = model.fit_generator(train_generator, steps_per_epoch = train_generator.samples // batch_size, validation_data = validation_generator,  validation_steps = validation_generator.samples // batch_size, epochs = n_epochs, callbacks=[LogRunMetrics()])        

score = model.evaluate_generator(test_generator, steps= test_generator.samples // batch_size)

# log a single value
run.log("Final test loss", score[0])
print('Test loss:', score[0])

run.log('Final test accuracy', score[1])
print('Test accuracy:', score[1])

# create a ./outputs/model folder in the compute target
# files saved in the "./outputs" folder are automatically uploaded into run history
os.makedirs('./outputs/model', exist_ok=True)

# serialize NN architecture to JSON
model_json = model.to_json()
# save model JSON
with open('./outputs/model/model.json', 'w') as f:
    f.write(model_json)
# save model weights
model.save_weights('./outputs/model/model.h5')
print("model saved in ./outputs/model folder")

Overwriting /mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/meetup-training/train.py


### Create an estimator

An estimator object is used to submit the run. Azure Machine Learning has pre-configured estimators for common machine learning frameworks, as well as generic Estimator. Create Keras estimator by specifying

* The name of the estimator object, `est`
* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will use the AmlCompute you created
* The training script name, train.py
* Parameters required from the training script 

In this tutorial, this target is am-ml-compute. All files in the script folder are uploaded into the cluster nodes for execution. The data_folder is set to use the datastore (`ds.path('mnist').as_mount()`).

In [83]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': ds.path('dataset').as_mount()
}

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target, 
                 pip_packages=['keras', 'matplotlib', 'pillow'],
                 entry_script='train.py')

WARNING - framework_version is not specified, defaulting to version 1.13.


This is what the mounting point looks like:

In [84]:
print(ds.path('dataset').as_mount())

$AZUREML_DATAREFERENCE_eb00256191c64fd483f01606d54abdfe


### Submit the job to the cluster

Run the experiment by submitting the estimator object. And you can navigate to Azure portal to monitor the run.

In [85]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
azuremeetup-ml-exp,azuremeetup-ml-exp_1565735022_c4a7d528,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


Since the call is asynchronous, it returns a **Preparing** or **Running** state as soon as the job is started.

## Monitor a remote run

In total, the first run takes **approximately 10 minutes**. But for subsequent runs, as long as the dependencies (`conda_packages` parameter in the above estimator constructor) don't change, the same image is reused and hence the container start up time is much faster.

Here is what's happening while you wait:

- **Image creation**: A Docker image is created matching the Python environment specified by the estimator. The image is built and stored in the ACR (Azure Container Registry) associated with your workspace. Image creation and uploading takes **about 5 minutes**. 

  This stage happens once for each Python environment since the container is cached for subsequent runs.  During image creation, logs are streamed to the run history. You can monitor the image creation progress using these logs.

- **Scaling**: If the remote cluster requires more nodes to execute the run than currently available, additional nodes are added automatically. Scaling typically takes **about 5 minutes.**

- **Running**: In this stage, the necessary scripts and files are sent to the compute target, then data stores are mounted/copied, then the entry_script is run. While the job is running, stdout and the files in the ./logs directory are streamed to the run history. You can monitor the run's progress using these logs.

- **Post-Processing**: The ./outputs directory of the run is copied over to the run history in your workspace so you can access these results.


You can check the progress of a running job in multiple ways. This tutorial uses a Jupyter widget as well as a `wait_for_completion` method. 

### Jupyter widget

Watch the progress of the run with a Jupyter widget.  Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [86]:
from azureml.widgets import RunDetails
RunDetails(run).show()

A Jupyter Widget

By the way, if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

### Get log results upon completion

Model training happens in the background. You can use `wait_for_completion` to block and wait until the model has completed training before running more code. 

In [ ]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=True) 

### Display run results

You now have a model trained on a remote cluster.  Retrieve all the metrics logged during the run, including the accuracy of the model:

In [12]:
print(run.get_metrics())

{'Loss': [0.6591263505674544, 0.6024643259606083, 0.5557346298419374, 0.5136320827336147, 0.48925342275642386, 0.46613009490709434, 0.45246047112180376, 0.43646186871685905, 0.4007196307316355, 0.39871538815827207, 0.39563786429443815, 0.36860971901191586, 0.3653653468238539, 0.3554818710197752, 0.33730984142963316, 0.3455456301592279, 0.3222096601704965, 0.29538040548965133, 0.3069666387929373, 0.2813778810504673], 'Accuracy': [0.6132192622338023, 0.6821589438632868, 0.7188905766163988, 0.7529985197837921, 0.7666167039563809, 0.7839830201426368, 0.7812343988640198, 0.7947276459819731, 0.8184657683973012, 0.8189655250158981, 0.8158420864967392, 0.8344577769825662, 0.836706656566088, 0.8400799739307192, 0.8501999068117213, 0.8432034061945182, 0.857071472250897, 0.8716891620648854, 0.8605697189850071, 0.8775612313350637], 'Final test loss': 0.5011591619771459, 'Final test accuracy': 0.805027186870575}


In the next tutorial you will explore this model in more detail.

## Register model

The last step in the training script wrote the trained model to `outputs/model` in the VM of the cluster where the job is executed. `outputs` is a special directory in that all content in this  directory is automatically uploaded to your workspace.  This content appears in the run record in the experiment under your workspace. Hence, the model file is now also available in your workspace.

You can see files associated with that run.

In [13]:
print(run.get_file_names())

['azureml-logs/55_batchai_execution.txt', 'azureml-logs/55_batchai_stdout-job_post.txt', 'azureml-logs/55_batchai_stdout-job_prep.txt', 'azureml-logs/55_batchai_stdout.txt', 'azureml-logs/56_batchai_stderr.txt', 'azureml-logs/70_driver_log.txt', 'logs/azureml/146_azureml.log', 'logs/azureml/azureml.log', 'outputs/model/model.h5', 'outputs/model/model.json']


Register the model in the workspace so that you (or other collaborators) can later query, examine, and deploy this model.

In [88]:
# register model 
model = run.register_model(model_name='meetup-ml-model', model_path='outputs/model')
print(model.name, model.id, model.version, sep='\t')

meetup-ml-model	meetup-ml-model:3	3


![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/img-classification-part1-training.png)

## Deploy as a web service

After you tested the model and you're satisfied with the results, deploy the model as a web service hosted in Container Instances.

To build the correct environment for Container Instances, provide the following components:
- A scoring script to show how to use the model.
- An environment file to show what packages need to be installed.
- A configuration file to build the container instance.
- The model you trained previously.


### Fetch the model

In [89]:
model = Model(ws, 'meetup-ml-model')

model.download(target_dir=os.getcwd(), exist_ok=True)

'/mnt/azmnt/code/Users/live.com#andrew.fletcher16/meetup/model'

### Define the scoring script

In [100]:
%%writefile score.py

import json
import numpy as np
import os
import pickle
import argparse
import base64
import io

import matplotlib.pyplot as plt

import keras
from keras.models import Sequential, model_from_json, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Activation
from keras.optimizers import RMSprop
from keras.callbacks import Callback
from keras.preprocessing.image import ImageDataGenerator
import imageio
import cv2
import json

import tensorflow as tf

from azureml.core import Run

from azureml.core.model import Model

from azureml.contrib.services.aml_response import AMLResponse

def init():
    global model
    
    model_path = Model.get_model_path('meetup-ml-model')
    json_model_path = os.path.join(model_path, 'model.json')
    weights_path = os.path.join(model_path, 'model.h5')

    json_model_file = open(json_model_path, "r")
    loaded_model_json = json_model_file.read()
    json_model_file.close()
    
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weights_path)
    
    model = loaded_model  


def run(raw_data):     
    img_size = 150
    
    img = imageio.imread(io.BytesIO(base64.b64decode(raw_data)))
    
    r_img = cv2.resize(img, dsize=(img_size, img_size), interpolation=cv2.INTER_CUBIC)
    
    data = np.array(r_img, dtype=np.float32)    
    
    r_data = np.reshape(data, (1,150,150,3))
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow(r_data, batch_size=1)
    
    res = model.predict_generator(test_generator, steps=1)
    
    json_res = json.dumps(res.tolist())

    resp = AMLResponse(json_res, 200, json_str=True)
    resp.headers['Access-Control-Allow-Origin'] = "*"
    
    return resp
    
    

Overwriting score.py


### Define the Environment Config

In [101]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies()
myenv.add_pip_package("numpy")
myenv.add_pip_package("azureml-core")
myenv.add_pip_package("tensorflow")
myenv.add_pip_package("keras")
myenv.add_pip_package("matplotlib")
myenv.add_pip_package("pillow")
myenv.add_pip_package("imageio")
myenv.add_pip_package("opencv-python-headless")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

### Define a Deployment config

In [102]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               tags={"data": "CatMeerkatDog",
                                                     "method": "keras-dlnn"},
                                               description='Predict types of animal with a DLNN in Keras')

### Create an image and deploy it

In [103]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='meetup-ml-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Running..................................................
Succeeded
Image creation operation finished for image meetup-ml-svc:13, operation "Succeeded"
Creating service
Running.....................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 2.56 s, sys: 192 ms, total: 2.75 s
Wall time: 6min 6s


In [104]:
print(service.scoring_uri)

http://acf26876-6fed-4660-be67-ea9d77513aee.northeurope.azurecontainer.io/score
